In [1]:
import os
import sys
import csv

import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# list shp files recursively
os.chdir('..')
abs_path = os.getcwd()

HIFLD_path = os.path.join(abs_path, 'data/HIFLD')
shp_files = [os.path.join(root, name) \
             for root, dirs, files in os.walk(HIFLD_path) \
             for name in files \
             if name.endswith(('.shp'))]

In [3]:
# number of observations per geoshape file 
for file in shp_files:
    with open(file, 'rb') as f:
        row_count = len(f.readlines()) - 1
        
        basename = os.path.basename(file).split('/')[0]
        fname = os.path.basename(basename).split('.')[0]
        
        print('There are {} observations in {}.'.format(row_count, fname))

There are 267622 observations in AllPlacesOfWorship.
There are 92960 observations in Alternative_Fueling_Stations.
There are 7167 observations in CollegesUniversities.
There are 93348 observations in FDIC_Insured_Banks.
There are 54785 observations in Fire_Stations.
There are 8655 observations in Hospitals.
There are 36503 observations in NCUA_Insured_Credit_Unions_V_2024_Q1.
There are 66279 observations in Pharmacies_.
There are 5056 observations in Prison_Boundaries.
There are 24089 observations in Private_Schools.
There are 112783 observations in PublicSchools.
There are 5298 observations in UrgentCareFacs.


# Explore Geoshape Files
* Columns
* Data types
* CRS

In [4]:
def load2gdf_slices(in_fc, use_fields=None, chunk_size=1000):
    """
    Iteratively slice columns from a GeoDataFrame

    Parameters
    ----------
    in_fc (str): path to geoshapefile
    use_fields (list): column names to slice
    chunk_size (int): number of rows per iteration
    """
    i = 0
    out_gdf = gpd.GeoDataFrame()
    gdf = []
    f_geom = 'geometry'
    
    #import pdb; pdb.set_trace()
    while True:
        chunk = slice(i, i + chunk_size, 1)
        gdf_iter = gpd.read_file(in_fc, rows=chunk)
    
        if gdf_iter.shape[0] == 0:
            break
        else:
            if use_fields:
                if f_geom not in use_fields:
                    #use_fields.append('geometry')
                    pass
                gdf_iter = gdf_iter[use_fields]
    
            gdf.append(gdf_iter)
            i += chunk_size
        gdf_concat = pd.concat(gdf)
        out_gdf = gpd.GeoDataFrame(gdf_concat)
    return out_gdf

In [5]:
%%time

AllPlacesOfWorship_gdf = load2gdf_slices(shp_files[0], use_fields=['STATE', 'NAME', 'STREET', 'CITY', 'geometry', 'ZIP'], chunk_size=1000)

CPU times: total: 2min 11s
Wall time: 5min 10s


In [16]:
allplacesofworship_address = full_address(AllPlacesOfWorship_gdf)

allplacesofworship_states_EPSG4326 = allplacesofworship_address.to_crs("EPSG:4326")
allplacesofworship_states_EPSG4326['source_centroid'] = allplacesofworship_states_EPSG4326['geometry'].centroid
allplacesofworship_states_EPSG4326['source_lon'] = allplacesofworship_states_EPSG4326['geometry'].centroid.x
allplacesofworship_states_EPSG4326['source_lat'] = allplacesofworship_states_EPSG4326['geometry'].centroid.y
allplacesofworship_states_EPSG4326['Place_type'] = 'AllPlacesOfWorship'

allplacesofworship_states_EPSG4326 = keep_columns(allplacesofworship_states_EPSG4326)

C:\Users\bchan\AppData\Local\Temp\ipykernel_20536\322628928.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  allplacesofworship_states_EPSG4326['source_centroid'] = allplacesofworship_states_EPSG4326['geometry'].centroid
C:\Users\bchan\AppData\Local\Temp\ipykernel_20536\322628928.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  allplacesofworship_states_EPSG4326['source_lon'] = allplacesofworship_states_EPSG4326['geometry'].centroid.x
C:\Users\bchan\AppData\Local\Temp\ipykernel_20536\322628928.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  allplacesofworship_s

In [20]:
shp_file = allplacesofworship_states_EPSG4326.set_geometry('source_centroid')

save_dir = os.path.join(abs_path, 'output/HIFLD/centroids')
save_path = os.path.join(save_dir, 'AllPlacesofWorship_full')
create_dir(save_path)

shp_file.to_file(save_path, driver='ESRI Shapefile')

C:\Users\bchan\AppData\Local\Temp\ipykernel_20536\1806490047.py:7: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_file.to_file(save_path, driver='ESRI Shapefile')


In [6]:
%%time

PublicSchools_gdf = load2gdf_slices(shp_files[10], use_fields=['STATE', 'NAME', 'ADDRESS', 'CITY', 'ZIP', 'geometry'], chunk_size=2000)

CPU times: total: 28.4 s
Wall time: 1min 1s


In [ ]:
publicschools_address = full_address(PublicSchools_gdf)

publicschools_address_EPSG4326 = publicschools_address.to_crs("EPSG:4326")
publicschools_address_EPSG4326['source_centroid'] = publicschools_address_EPSG4326['geometry'].centroid
publicschools_address_EPSG4326['source_lon'] = publicschools_address_EPSG4326['geometry'].centroid.x
publicschools_address_EPSG4326['source_lat'] = publicschools_address_EPSG4326['geometry'].centroid.y
publicschools_address_EPSG4326['Place_type'] = 'PublicSchools'

publicschools_address_EPSG4326 = keep_columns(publicschools_address_EPSG4326)

In [ ]:
shp_file = publicschools_address_EPSG4326.set_geometry('source_centroid')

save_dir = os.path.join(abs_path, 'output/HIFLD/centroids')
save_path = os.path.join(save_dir, 'PublicSchools_full')
create_dir(save_path)

shp_file.to_file(save_path, driver='ESRI Shapefile')

In [7]:
def read_shp(file, rows=100):
    """
    Read geoshapes file

    Parameters
    ----------
        rows (int): number of rows per file to read

    Returns
    -------
        df (GeoDataFrame)
    """
    df = gpd.read_file(file, rows=rows)

    return df

In [8]:
# read all geoshape files into dictionary
dict_geo = {}
for file in shp_files:
    basename = os.path.basename(file).split('/')[0]
    fname = os.path.basename(basename).split('.')[0]

    # print(file)
    dict_geo[fname] = read_shp(file, rows=6000)

## Columns

In [6]:
dict_geo['FDIC_Insured_Banks'].columns

Index(['OBJECTID', 'ADDRESBR', 'BRNUM', 'BRSERTYP', 'CBSABR', 'CBSANAMB',
       'CITYBR', 'CNTRYNAB', 'CNTYNAMB', 'DEPSUMBR', 'GEOCODE_CE', 'NAMEBR',
       'STALPBR', 'STCNTYBR', 'STNAMEBR', 'UNINUMBR', 'ZIPBR', 'CERT',
       'ADDRESS', 'ASSET', 'BKCLASS', 'CITY', 'CNTRYNA', 'DENOVO', 'DEPDOM',
       'NAMEFULL', 'NAMEHCR', 'REGAGNT', 'REPDTE', 'RSSDID', 'STALP', 'STCNTY',
       'STNAME', 'ZIP', 'BKMO', 'LOC_NAME', 'STATUS', 'SCORE', 'x', 'y',
       'GeocodeSou', 'STD_ADDR_B', 'STD_ADDR', 'ZIP4BR', 'geometry'],
      dtype='object')

In [7]:
dict_geo['Prison_Boundaries'].columns

Index(['FID', 'FACILITYID', 'NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'ZIP4',
       'TELEPHONE', 'TYPE', 'STATUS', 'POPULATION', 'COUNTY', 'COUNTYFIPS',
       'COUNTRY', 'NAICS_CODE', 'NAICS_DESC', 'SOURCE', 'SOURCEDATE',
       'VAL_METHOD', 'VAL_DATE', 'WEBSITE', 'SECURELVL', 'CAPACITY',
       'SHAPE_Leng', 'GlobalID', 'CreationDa', 'Creator', 'EditDate', 'Editor',
       'SHAPE_Le_1', 'SHAPE_Area', 'geometry'],
      dtype='object')

In [8]:
dict_geo['Fire_Stations'].columns

Index(['OBJECTID', 'PERMANENT_', 'SOURCE_FEA', 'SOURCE_DAT', 'SOURCE_D_1',
       'SOURCE_ORI', 'DATA_SECUR', 'DISTRIBUTI', 'LOADDATE', 'FTYPE', 'FCODE',
       'NAME', 'ISLANDMARK', 'POINTLOCAT', 'ADMINTYPE', 'ADDRESSBUI',
       'ADDRESS', 'CITY', 'STATE', 'ZIPCODE', 'GNIS_ID', 'FOOT_ID',
       'COMPLEX_ID', 'GLOBALID', 'geometry'],
      dtype='object')

## CRS

In [9]:
dict_geo['FDIC_Insured_Banks'].crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [10]:
dict_geo['Prison_Boundaries'].crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

# Full Address String
* Output full address string (Street, City, State ZIP) for each GeoDataFrame

In [15]:
def full_address(df):
    """
    Concatenate each address component to a full address string
         (Street, City, State ZIP)
         
    Parameters
    ----------
        df (GeoDataFrame)

    Returns
    -------
        df (GeoDataFrame): with 'full_address' column
    """
    if 'ZIP' in df.columns:
        zip_col = 'ZIP'
    elif 'ZIPCODE' in df.columns:
        zip_col = 'ZIPCODE'
    elif 'zip' in df.columns:
        zip_col = 'zip'

    if 'STATE' in df.columns:
        state_col = 'STATE'
    elif 'state' in df.columns:
        state_col = 'state'
    else:
        state_col = 'STNAME'

    if 'ADDRESS' in df.columns:
        addr_col = 'ADDRESS'
    elif 'street_add' in df.columns:
        addr_col = 'street_add'
    elif 'ADDRESS' not in df.columns:
        addr_col = 'STREET'
    elif ('STREET' not in df.columns) and ('ADDRESS' not in df.columns):
        addr_col = 'STD_ADDR_B'

    if 'CITY' in df.columns:
        city_col = 'CITY'
    elif 'city' in df.columns:
        city_col = 'city'

    # add 0 to zip code if only 4 digits
    df['zip'] = df[zip_col].astype('str')
    df['zip'] = df['zip'].apply(lambda x: '0' + x if len(x) <5 else x)
    
    df[addr_col] = df[addr_col].astype('str')
    df[city_col] = df[city_col].astype('str')
    df[state_col] = df[state_col].astype('str')
    
    df['Full_Address'] = df[[addr_col, city_col, state_col]].fillna('NaN').agg(', '.join, axis=1) + ' ' + df['zip']
    df['Full_Address'] = df['Full_Address'].astype('str')
             
    return df

In [10]:
%%time
dict_address = {}
for fname in dict_geo:
    print(fname)
    dict_address[fname] = full_address(dict_geo[fname])

AllPlacesOfWorship
Alternative_Fueling_Stations
CollegesUniversities
FDIC_Insured_Banks
Fire_Stations
Hospitals
NCUA_Insured_Credit_Unions_V_2024_Q1
Pharmacies_
Prison_Boundaries
Private_Schools
PublicSchools
UrgentCareFacs
CPU times: total: 328 ms
Wall time: 577 ms


# Calculate Latitutde / Longitude
1. Convert each GeoDataFrame to 'EPSG:4326'
2. Calculate centroids from geometry in each GeoDataFrame
3. Extract latitude and longitude from each centroid

In [11]:
def convert_EPSG4326(dict):
    """
    Convert each GeoDataFrame to 'EPSG:4326'
         
    Parameters
    ----------
        dict (dictionary): of GeoDataFrames

    Returns
    -------
        dict (dictionary): GeoDataFrames of 'EPSG:4326' CRS
    """
    for fname in dict:
        dict[fname] = dict[fname].to_crs("EPSG:4326")

    return dict

In [12]:
def get_centroid(dict):
    """
    Convert each GeoDataFrame to 'EPSG:4326'
         
    Parameters
    ----------
        dict (dictionary): GeoDataFrames of 'EPSG:4326' CRS

    Returns
    -------
        dict (dictionary): GeoDataFrames with extracted centroids from geoshapes
    """
    dict_centroids = {}
    for fname in dict:
        if 'x' in dict[fname].columns:
            dict[fname] = dict[fname].rename(columns={'x': 'source_lon', 'y': 'source_lat'})
            dict[fname]['Place_type'] = os.path.basename(fname)
            dict[fname]['source_centroid'] = dict[fname]['geometry']
        else:
            dict[fname]['source_centroid'] = dict[fname]['geometry'].centroid
            dict[fname]['source_lon'] = dict[fname]['geometry'].centroid.x
            dict[fname]['source_lat'] = dict[fname]['geometry'].centroid.y
            dict[fname]['Place_type'] = os.path.basename(fname)

        dict_centroids[fname] = keep_columns(dict[fname])
        
    return dict_centroids

In [13]:
def keep_columns(df):
    """
    Convert each GeoDataFrame to 'EPSG:4326'
         
    Parameters
    ----------
        df (GeoDataFrame): GeoDataFrames of 'EPSG:4326' CRS with extracted centroids

    Returns
    -------
        new_df (GeoDataFrame): GeoDataFrames with only source centroid and full address columns
    """
    cols = ['Full_Address', 'Place_type', 'source_centroid', 'source_lon', 'source_lat']
    new_df = df[cols]

    return new_df

In [14]:
# if you get an error on the first pass, try re-running the cell
dict_EPSG4326 = convert_EPSG4326(dict_address)
dict_centroid = get_centroid(dict_EPSG4326)

C:\Users\bchan\AppData\Local\Temp\ipykernel_20536\4286707561.py:20: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  dict[fname]['source_centroid'] = dict[fname]['geometry'].centroid
C:\Users\bchan\AppData\Local\Temp\ipykernel_20536\4286707561.py:21: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  dict[fname]['source_lon'] = dict[fname]['geometry'].centroid.x
C:\Users\bchan\AppData\Local\Temp\ipykernel_20536\4286707561.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  dict[fname]['source_lat'] = dict[fname]['geometry'].centroid.y
C:\Users\bchan\AppData\Local\Temp\ipykernel

In [29]:
# check centroids
dict_centroid['FDIC_Insured_Banks']

,Full_Address,Place_type,source_centroid,source_lon,source_lat
0,"100 North Tryon St, Charlotte, North Carolina ...",FDIC_Insured_Banks,POINT (-72.87865 41.60144),-72.878648,41.601435
1,"100 North Tryon St, Charlotte, North Carolina ...",FDIC_Insured_Banks,POINT (-70.25667 43.65743),-70.256668,43.657432
2,"2 Elm Street, Camden, Maine 04843",FDIC_Insured_Banks,POINT (-68.42462 44.54133),-68.424621,44.541326
3,"1 Lincoln St. Fl 1, Boston, Massachusetts 02111",FDIC_Insured_Banks,POINT (-71.05796 42.35282),-71.057959,42.352824
4,"100 North Tryon St, Charlotte, North Carolina ...",FDIC_Insured_Banks,POINT (-71.11891 42.37383),-71.118913,42.373832
...,...,...,...,...,...
5995,"1039 West Mitchell Street, Milwaukee, Wisconsi...",FDIC_Insured_Banks,POINT (-87.92526 43.01226),-87.925261,43.012262
5996,"40214 S Ekern Avenue, Pigeon Falls, Wisconsin ...",FDIC_Insured_Banks,POINT (-91.21058 44.42560),-91.210581,44.425597
5997,"210 East Main Street, Waupun, Wisconsin 53963",FDIC_Insured_Banks,POINT (-88.67650 43.80840),-88.676503,43.808401
5998,"101 South Main Street, Viroqua, Wisconsin 54665",FDIC_Insured_Banks,POINT (-90.88928 43.55612),-90.889284,43.556120


# Save Geoshape Files

In [18]:
def save_shp(dict, save_dir):
    """
    Save each GeoDataFrame to individual geoshape files
         
    Parameters
    ----------
        dict (dictionary): GeoDataFrames of 'EPSG:4326' CRS with extracted centroids
        save_dir (str): path of desired output directory
    """
    for fname in dict:
        shp_file = dict[fname].set_geometry('source_centroid')
        
        save_path = os.path.join(save_dir, f"{fname}")
        create_dir(save_path)
        
        shp_file.to_file(save_path, driver='ESRI Shapefile')

In [19]:
def create_dir(save_dir):
    """
    Creates directory if it does not exist
         
    Parameters
    ----------
        save_dir (str): path of desired output directory
    """
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

In [32]:
# save files
save_dir = os.path.join(abs_path, 'output/HIFLD/centroids')
create_dir(save_dir)

save_shp(dict_centroid, save_dir)

C:\Users\bchan\AppData\Local\Temp\ipykernel_23184\2389177360.py:16: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_file.to_file(save_path, driver='ESRI Shapefile')
C:\Users\bchan\AppData\Local\Temp\ipykernel_23184\2389177360.py:16: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_file.to_file(save_path, driver='ESRI Shapefile')
C:\Users\bchan\AppData\Local\Temp\ipykernel_23184\2389177360.py:16: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_file.to_file(save_path, driver='ESRI Shapefile')
C:\Users\bchan\AppData\Local\Temp\ipykernel_23184\2389177360.py:16: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_file.to_file(save_path, driver='ESRI Shapefile')
C:\Users\bchan\AppData\Local\Temp\ipykernel_23184\2389177360.py:16: UserWarning: Column names longer than 10

In [22]:
gpd_concat = pd.concat([dict_centroid[fname] for fname in dict_centroid])
gpd_concat = pd.concat([gpd_concat, allplacesofworship_states_EPSG4326, publicschools_address_EPSG4326])
gpd_concat

,Full_Address,Place_type,source_centroid,source_lon,source_lat
0,"523 E BROADWAY, SOUTH BOSTON, MA 02127",AllPlacesOfWorship,POINT (-71.04352 42.33547),-71.043522,42.335472
1,"454 ESSEX ST, LAWRENCE, MA 01840",AllPlacesOfWorship,POINT (-71.16494 42.70621),-71.164940,42.706213
2,"569 BROADWAY, NEWARK, NJ 07104",AllPlacesOfWorship,POINT (-74.16282 40.76993),-74.162821,40.769935
3,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",AllPlacesOfWorship,POINT (-78.74782 42.79853),-78.747816,42.798535
4,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",AllPlacesOfWorship,POINT (-70.11378 44.42861),-70.113777,44.428610
...,...,...,...,...,...
1294,"115 STEWARTS FERRY PIKE, NASHVILLE, TN 37214",PublicSchools,POINT (-86.65375 36.16762),-86.653751,36.167621
1295,"1250 VOLLINTINE AVE, MEMPHIS, TN 38107",PublicSchools,POINT (-90.01485 35.16550),-90.014851,35.165500
1296,"1250 VOLLINTINE AVE, MEMPHIS, TN 38107",PublicSchools,POINT (-90.01469 35.16544),-90.014688,35.165440
1297,"2610 CINEMA DR, MARYVILLE, TN 37804",PublicSchools,POINT (-83.94173 35.79683),-83.941730,35.796833


In [23]:
gpd_concat.to_csv(os.path.join(save_dir, "centroids_432k.csv"), index=False)